In [2]:
import numpy as np
import pandas as pd
from numpy.random import rand
import gzip
import json
import ast
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt
import random
from scipy.sparse import coo_matrix
from scipy.sparse import lil_matrix
from numpy.random import rand
from scipy.sparse import csr_matrix, isspmatrix_csr
from pandas.api.types import CategoricalDtype
#df = getDF('reviews_Amazon_Instant_Video.json.gz')[['reviewerID','asin']]
#df  is the whole table of data, we need to act on this
# very imp code : https://stackoverflow.com/questions/49405101/grouping-all-rows-of-a-pandas-dataframewith-many-columns-with-the-same-value-i
#df = df[['reviewerID','asin']]
#df1 = df.astype(str).groupby('asin').agg(','.join).reset_index()
#df1

In [3]:
alpha_network_data = pd.read_csv("alpha_network.csv")
alpha_network_data.columns=["src", "Dest", "Rating", "Timestamp"]
alpha_network_data = alpha_network_data[["src", "Dest"]]
#alpha_network_data1 = alpha_network_data.astype(str).groupby("Dest").agg(','.join).reset_index()
alpha_network_data

,src,Dest
0,430,1
1,3134,1
2,3026,1
3,3010,1
4,804,1
5,160,1
6,95,1
7,377,1
8,888,1
9,89,1


In [4]:
alpha_gt_data = pd.read_csv("alpha_gt.csv")
alpha_gt_data.columns = ["ID","Label"]
alpha_gt_data.groupby('Label').count()

,ID
Label,
-1,102
1,137


In [5]:
alpha_IDs = alpha_gt_data['ID'].tolist()
len(alpha_IDs)

239

In [6]:
alpha_data_for_matrix = alpha_network_data.loc[alpha_network_data['src'].isin(alpha_IDs)]# filtered alpha_network_data
alpha_data_for_matrix

,src,Dest
6,95,1
13,351,1
19,37,1
21,821,1
28,20,1
37,118,1
38,3254,1
40,112,1
41,11,1
44,15,1


In [7]:
df = alpha_data_for_matrix
#df  is the whole table of data, we need to act on this
# very imp code : https://stackoverflow.com/questions/49405101/grouping-all-rows-of-a-pandas-dataframewith-many-columns-with-the-same-value-i

df_copy = df.copy()
df_copy["value"] = 1
df_copy# this prints the best col format. Now to convert to matrix:

reviewer = CategoricalDtype(sorted(df_copy["src"].unique()), ordered=True)
product = CategoricalDtype(sorted(df_copy["Dest"].unique()), ordered=True)

row = df_copy["src"].astype(reviewer).cat.codes
col = df_copy["Dest"].astype(product).cat.codes

sparse_matrix = csr_matrix((df_copy["value"], (row, col)),
                           shape=(reviewer.categories.size, product.categories.size))
M = sparse_matrix.toarray() #prints the matrix
#A.tocsr()[:,select_ind].todense()
#sparse_matrix.get_shape()
M.shape

(214, 2445)

In [9]:
A = M
k = 100
l = k+20
m = 214
n = 2445
# now in the first step we choose k = 100, l = 102, i = 2, A is considered mxn , so m = 10000, n= 5000
# In the first step, Using a random number generator, form a real n × l matrix G whose entries are independent and identically distributed
# Gaussian random variables of zero mean and unit variance
mu = 0
sigma = 1.0
G = np.random.normal(mu, sigma, (n,l))
#Compute B = AG (B ∈ R^ m×l )
#B = np.matmul(A,G)#---------------------------------------------------------first multiplication by A
B = []
for i in range(l):
    v = G[:,i]
    x = A.dot(v)
    B.append(x)
B = np.array(B).T
A = None # no more needed
G = None# not reqd anymore
X, lamda, Yt = np.linalg.svd(B, full_matrices=True)

lamda = None
Yt =  None
Q = X[:, : k] #(Q ∈ R m×k )
#Qt = Q.transpose()

#T = np.matmul(Qt,A)#-------------------------------2nd mult by A, only these two are there in this algo
#Newsparse_matrix = csr_matrix((df_copy["value"], (col, row)),shape=(reviewer.categories.size, product.categories.size))
#At = coo_matrix((data, (col, row)), shape=(1000, 1000)).tocsr()
At = M.transpose()
T = []
for i in range(k):
    v = Q[:,i]
    x = At.dot(v)
    T.append(x)
T = np.array(T)
At = None# not needed anymore
row = None
col = None
W, singlr, Vt = np.linalg.svd(T, full_matrices=True)
T = None# not needed anymore
Vt = None# not needed for the smaller dim matrix creation
U = np.matmul(Q,W)
Q = None
W = None
final = np.multiply(U,singlr)
final

array([[  7.77949637e+00,   2.02042758e+01,  -7.19948707e-01, ...,
         -9.10438970e-05,  -8.13451651e-03,   5.15359804e-04],
       [  6.82391587e+00,  -1.09094027e+00,   6.61780786e+00, ...,
         -4.53162715e-02,   7.56372432e-03,  -4.31137350e-03],
       [  7.87402891e+00,  -5.45641472e-01,  -6.70675585e+00, ...,
          9.61666376e-03,  -3.53841614e-02,   6.50703720e-03],
       ..., 
       [  8.06619533e-01,  -1.53650161e-01,  -1.24728504e-01, ...,
         -5.91314084e-03,  -3.26182975e-02,  -6.64627612e-02],
       [  2.12327769e+00,   2.00136348e-01,   2.38680182e+00, ...,
         -3.20834593e-02,  -9.33992652e-03,  -1.72813672e-02],
       [  1.20317827e+00,  -3.10167844e-01,   3.72335767e-01, ...,
          1.33442012e-01,   1.43580620e-01,   4.04985251e-02]])

In [10]:
A = final
#Now do the 2-means and get the output.
#centroid, label = kmeans2(A, 2, minit='points')
#centroid will be two arrays each of 10 dimension for the two clusters
# label[i] is the code or index of the centroid the ith observation is closest to.

# Now we write the function that partitons A into A0 and A1
# A0 and A1 are lists which will contain only the indices of the rows which are supposed to be in A0 and A1
# partition takes an arr which means it contains the matrix made up of A[arr[i]]
# we need to form matrix from the rows of A as mentioned in arr
arr = list(range(214))# later this should be list(range(10**6))
def partition(arr):
    matrix = []
    for i in range(len(arr)):
        matrix.append(A[i,:])
    matrix = np.array(matrix)
    centroid, label = kmeans2(matrix, 2, minit='points')
    A0,A1 = [],[]
    for i in range(len(arr)):# this is the num of the rows, i.e. the num of accts
        if label[i] == 0:
            A0.append(i)# means A[i] should be in this as a row
        else:
            A1.append(i)# means A[i] should be in this as a row
    #A0 = np.array(A0)
    #A1 = np.array(A1)
    return (A0,A1)# returns two lists so that x = A0[i], A[x] will give the reqd row

tau = 3
# data contains lists which contain indices i. A[i] will be the reqd rows, to start with data = [0,...,9]
class Node:
    def __init__ (self, data, parent = None):
        #self.width = width
        #self.height = height
        self.parent = parent
        self.children = []
        self.data = data

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        #if self.children == []: return
        numrow = len(self.data)
        if numrow <= tau:
            self.children = []
            return
        A1,A2 = partition(self.data)
        self.children = [Node(A1, self),Node(A2, self)]
        
    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    def sizeofNodes(self):
        s = [len(self.data)]
        for child in self.children:
            s.append(child.sizeofNodes())
        #print(s)
        return s
    
N = Node(list(range(214)))
N.splitUntilLevel (5)
print(N.sizeofNodes())

[214, [196, [177, [3], [174, [1], [173, [1], [172]]]], [19, [1], [18, [16, [1], [15]], [2]]]], [18, [1], [17, [8, [6, [4], [2]], [2]], [9, [8, [1], [7]], [1]]]]]


In [2]:
epinions_network_data = pd.read_csv("epinions_network.csv")
epinions_network_data.columns=["src", "Dest", "Rating", "Timestamp"]
epinions_network_data = epinions_network_data[["src", "Dest"]]
#epinions_network_data1 = epinions_network_data.astype(str).groupby("Dest").agg(','.join).reset_index()
epinions_network_data

,src,Dest
0,1312460676,2252443524
1,204358,2252443524
2,368725,2252443524
3,277629,2252443524
4,246386,2252443524
5,293732,2252443524
6,525858,2252443524
7,237120,2252443524
8,971935620,2252443524
9,295723,2252443524


In [3]:
epinions_gt_data = pd.read_csv("epinions_gt.csv")
epinions_gt_data.columns = ["ID","Label"]
epinions_gt_data.groupby('Label').count()

,ID
Label,
-1,1012
1,9291


In [4]:
epinions_IDs = epinions_gt_data['ID'].tolist()
len(epinions_IDs)

10303

In [5]:
data_for_matrix = epinions_network_data.loc[epinions_network_data['src'].isin(epinions_IDs)]# filtered epinions_network_data
data_for_matrix

,src,Dest
0,1312460676,2252443524
1,204358,2252443524
2,368725,2252443524
3,277629,2252443524
4,246386,2252443524
5,293732,2252443524
6,525858,2252443524
7,237120,2252443524
9,295723,2252443524
10,526497,2252443524


In [8]:
df = data_for_matrix
#df  is the whole table of data, we need to act on this
# very imp code : https://stackoverflow.com/questions/49405101/grouping-all-rows-of-a-pandas-dataframewith-many-columns-with-the-same-value-i

df_copy = df.copy()
df_copy["value"] = 1
df_copy# this prints the best col format. Now to convert to matrix:

reviewer = CategoricalDtype(sorted(df_copy["src"].unique()), ordered=True)
product = CategoricalDtype(sorted(df_copy["Dest"].unique()), ordered=True)

row = df_copy["src"].astype(reviewer).cat.codes
col = df_copy["Dest"].astype(product).cat.codes

sparse_matrix = csr_matrix((df_copy["value"], (row, col)),
                           shape=(reviewer.categories.size, product.categories.size))
M = sparse_matrix.toarray() #prints the matrix
#A.tocsr()[:,select_ind].todense()
#sparse_matrix.get_shape()
M.shape

(9045, 59317)

In [ ]:
A = M
k = 300
l = k+20
m = 9045
n = 59317
# now in the first step we choose k = 100, l = 102, i = 2, A is considered mxn , so m = 10000, n= 5000
# In the first step, Using a random number generator, form a real n × l matrix G whose entries are independent and identically distributed
# Gaussian random variables of zero mean and unit variance
mu = 0
sigma = 1.0
G = np.random.normal(mu, sigma, (n,l))
#Compute B = AG (B ∈ R^ m×l )
#B = np.matmul(A,G)#---------------------------------------------------------first multiplication by A
B = []
for i in range(l):
    v = G[:,i]
    x = A.dot(v)
    B.append(x)
B = np.array(B).T
A = None # no more needed
G = None# not reqd anymore
X, lamda, Yt = np.linalg.svd(B, full_matrices=True)

lamda = None
Yt =  None
Q = X[:, : k] #(Q ∈ R m×k )
#Qt = Q.transpose()

#T = np.matmul(Qt,A)#-------------------------------2nd mult by A, only these two are there in this algo
#Newsparse_matrix = csr_matrix((df_copy["value"], (col, row)),shape=(reviewer.categories.size, product.categories.size))
#At = coo_matrix((data, (col, row)), shape=(1000, 1000)).tocsr()
At = M.transpose()
T = []
for i in range(k):
    v = Q[:,i]
    x = At.dot(v)
    T.append(x)
T = np.array(T)
At = None# not needed anymore
row = None
col = None
W, singlr, Vt = np.linalg.svd(T, full_matrices=True)
T = None# not needed anymore
Vt = None# not needed for the smaller dim matrix creation
U = np.matmul(Q,W)
Q = None
W = None
final = np.multiply(U,singlr)
final

In [ ]:
A = final
#Now do the 2-means and get the output.
#centroid, label = kmeans2(A, 2, minit='points')
#centroid will be two arrays each of 10 dimension for the two clusters
# label[i] is the code or index of the centroid the ith observation is closest to.

# Now we write the function that partitons A into A0 and A1
# A0 and A1 are lists which will contain only the indices of the rows which are supposed to be in A0 and A1
# partition takes an arr which means it contains the matrix made up of A[arr[i]]
# we need to form matrix from the rows of A as mentioned in arr
arr = list(range(9045))# later this should be list(range(10**6))
def partition(arr):
    matrix = []
    for i in range(len(arr)):
        matrix.append(A[i,:])
    matrix = np.array(matrix)
    centroid, label = kmeans2(matrix, 2, minit='points')
    A0,A1 = [],[]
    for i in range(len(arr)):# this is the num of the rows, i.e. the num of accts
        if label[i] == 0:
            A0.append(i)# means A[i] should be in this as a row
        else:
            A1.append(i)# means A[i] should be in this as a row
    #A0 = np.array(A0)
    #A1 = np.array(A1)
    return (A0,A1)# returns two lists so that x = A0[i], A[x] will give the reqd row

tau = 3
# data contains lists which contain indices i. A[i] will be the reqd rows, to start with data = [0,...,9]
class Node:
    def __init__ (self, data, parent = None):
        #self.width = width
        #self.height = height
        self.parent = parent
        self.children = []
        self.data = data

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        #if self.children == []: return
        numrow = len(self.data)
        if numrow <= tau:
            self.children = []
            return
        A1,A2 = partition(self.data)
        self.children = [Node(A1, self),Node(A2, self)]
        
    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    def sizeofNodes(self):
        s = [len(self.data)]
        for child in self.children:
            s.append(child.sizeofNodes())
        #print(s)
        return s
    
N = Node(list(range(9045)))
N.splitUntilLevel (5)
print(N.sizeofNodes())

In [6]:
otc_network_data = pd.read_csv("otc_network.csv")
otc_network_data.columns=["src", "Dest", "Rating", "Timestamp"]
#otc_network_data = otc_network_data[["src", "Dest"]]
#otc_network_data1 = otc_network_data.astype(str).groupby("Dest").agg(','.join).reset_index()
otc_network_data# this prints the df without filtering the value with gt

,src,Dest,Rating,Timestamp
0,6,5,2,1.289242e+09
1,1,15,1,1.289243e+09
2,4,3,7,1.289245e+09
3,13,16,8,1.289254e+09
4,13,10,8,1.289254e+09
5,7,5,1,1.289363e+09
6,2,21,5,1.289371e+09
7,2,20,5,1.289371e+09
8,21,2,5,1.289381e+09
9,21,1,8,1.289441e+09


In [1]:
otc_gt_data = pd.read_csv("otc_gt.csv")
otc_gt_data.columns = ["ID","Label"]
otc_gt_data.groupby('Label').count()

NameError: name 'pd' is not defined

In [15]:
otc_IDs = otc_gt_data['ID'].tolist()
len(otc_IDs)

315

In [14]:
print(otc_network_data.loc[otc_network_data['src'].isin(otc_IDs)])# filtered otc_network_data
#https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

        src  Dest  Rating     Timestamp
0         6     5       2  1.289242e+09
1         1    15       1  1.289243e+09
2         4     3       7  1.289245e+09
3        13    16       8  1.289254e+09
4        13    10       8  1.289254e+09
5         7     5       1  1.289363e+09
6         2    21       5  1.289371e+09
7         2    20       5  1.289371e+09
8        21     2       5  1.289381e+09
9        21     1       8  1.289441e+09
10       21    10       8  1.289441e+09
11       21     8       9  1.289441e+09
12       21     3       7  1.289442e+09
13       17     3       5  1.289442e+09
14       17    23       1  1.289490e+09
15       10     1       8  1.289556e+09
16       10     6       7  1.289556e+09
17       10    21       8  1.289556e+09
18       10     8       1  1.289556e+09
19       10    25      10  1.289556e+09
20       10     2       7  1.289556e+09
21       10     3       7  1.289556e+09
22        4    26       1  1.289642e+09
27        1     5       4  1.289711e+09
